## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps


# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bolshaya Glushitsa,RU,50.2406,43.5908,59.97,73,100,4.50,overcast clouds
1,1,Butaritari,KI,3.0707,172.7902,80.13,75,96,4.74,overcast clouds
2,2,Kaitangata,NZ,-46.2817,169.8464,40.82,94,29,3.62,scattered clouds
3,3,Asau,RO,46.4333,26.4000,47.32,92,100,0.72,overcast clouds
4,4,Albany,US,42.6001,-73.9662,78.24,67,10,1.99,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) 
                                    & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Butaritari,KI,3.0707,172.7902,80.13,75,96,4.74,overcast clouds
4,4,Albany,US,42.6001,-73.9662,78.24,67,10,1.99,clear sky
5,5,Mahebourg,MU,-20.4081,57.7000,72.25,83,75,13.80,broken clouds
6,6,Keti Bandar,PK,24.1447,67.4497,81.27,72,2,12.59,clear sky
7,7,Arraial Do Cabo,BR,-22.9661,-42.0278,73.26,78,0,9.22,clear sky
9,9,Guerrero Negro,MX,27.9769,-114.0611,80.56,56,14,9.57,few clouds
10,10,Kapaa,US,22.0752,-159.3190,82.72,76,20,12.66,few clouds
17,17,Rajsamand,IN,25.0667,73.8833,78.13,61,96,12.97,overcast clouds
18,18,Bethel,US,41.3712,-73.4140,80.19,60,0,4.61,clear sky
19,19,Bambous Virieux,MU,-20.3428,57.7575,72.21,83,75,13.80,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                404
City                   404
Country                400
Lat                    404
Lng                    404
Max Temp               404
Humidity               404
Cloudiness             404
Wind Speed             404
Current Description    404
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_clean_df = preferred_cities_df.dropna()
preferred_cities_clean_df.count()

City_ID                400
City                   400
Country                400
Lat                    400
Lng                    400
Max Temp               400
Humidity               400
Cloudiness             400
Wind Speed             400
Current Description    400
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Butaritari,KI,80.13,overcast clouds,3.0707,172.7902,
4,Albany,US,78.24,clear sky,42.6001,-73.9662,
5,Mahebourg,MU,72.25,broken clouds,-20.4081,57.7000,
6,Keti Bandar,PK,81.27,clear sky,24.1447,67.4497,
7,Arraial Do Cabo,BR,73.26,clear sky,-22.9661,-42.0278,
9,Guerrero Negro,MX,80.56,few clouds,27.9769,-114.0611,
10,Kapaa,US,82.72,few clouds,22.0752,-159.3190,
17,Rajsamand,IN,78.13,overcast clouds,25.0667,73.8833,
18,Bethel,US,80.19,clear sky,41.3712,-73.4140,
19,Bambous Virieux,MU,72.21,light rain,-20.3428,57.7575,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat=row['Lat']
    lng=row['Lng']
    params['location'] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print('Hotel not found... skipping')

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df['Hotel Name'] != '']
clean_hotel_df.count()

City                   378
Country                378
Max Temp               378
Current Description    378
Lat                    378
Lng                    378
Hotel Name             378
dtype: int64

In [10]:
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Butaritari,KI,80.13,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
5,Mahebourg,MU,72.25,broken clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
6,Keti Bandar,PK,81.27,clear sky,24.1447,67.4497,Haji laloo memon house
7,Arraial Do Cabo,BR,73.26,clear sky,-22.9661,-42.0278,Pousada Porto Praia
9,Guerrero Negro,MX,80.56,few clouds,27.9769,-114.0611,"Casa Laguna, Bed & Breakfast"
...,...,...,...,...,...,...,...
711,Beira,MZ,75.29,few clouds,-19.8436,34.8389,Beira Terrace Hotel
712,Maragogi,BR,78.91,clear sky,-9.0122,-35.2225,Salinas do Maragogi All Inclusive Resort
713,Korla,CN,74.26,overcast clouds,41.7597,86.1469,Gulisitan Hotel
714,Miraflores,MX,73.98,broken clouds,19.2167,-98.8083,Hotel Mansión


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)


# Call the figure to plot the data.
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))